In [5]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from scipy import sparse
from scipy.sparse import coo_matrix
from warnings import filterwarnings
import torch

filterwarnings('ignore')

# -----------------------
# RBF Kernel
# -----------------------
def rbf(X1, X2, **kwargs):
    return np.exp(-cdist(X1, X2) ** 2 * kwargs['gamma'])

# -----------------------
# Graph Structure Learning Parameters
# -----------------------
GRAPH_SEED = 11
GRAPH_LEARN_MEAN = 0.5
GRAPH_LEARN_STD = 0.01
GRAPH_LEARN_EPOCHS = 5000
GRAPH_LEARN_ALPHA = 1.0
GRAPH_LEARN_BETA = 2.0
GRAPH_LEARN_ETA = 0.001

# -----------------------
# Graph Structure Learning
# -----------------------
def learn_graph(X: torch.Tensor, seed_graph: int) -> np.ndarray:
    X_np = X.cpu().numpy()
    N = X_np.shape[0]

    rng = np.random.default_rng(seed_graph)
    W = rng.normal(GRAPH_LEARN_MEAN, GRAPH_LEARN_STD, size=(N, N))

    i_idx, j_idx = np.triu_indices(N, k=1)
    w = W[i_idx, j_idx].copy()
    delL = np.sum((X_np[i_idx] - X_np[j_idx]) ** 2, axis=1)

    row = np.concatenate([i_idx, j_idx])
    col = np.concatenate([np.arange(len(i_idx)), np.arange(len(j_idx))])
    S = coo_matrix((np.ones(len(row)), (row, col)), shape=(N, len(i_idx))).tocsc()
    S_T = S.T

    best_w = w.copy()
    best_loss = np.inf

    for _ in range(GRAPH_LEARN_EPOCHS):
        Sw = S.dot(w)
        inv_Sw = np.zeros_like(Sw)
        nonzero = Sw > 0
        inv_Sw[nonzero] = 1.0 / Sw[nonzero]

        grad = 2.0 * delL + GRAPH_LEARN_BETA * w - GRAPH_LEARN_ALPHA * S_T.dot(inv_Sw)
        w -= GRAPH_LEARN_ETA * grad
        np.clip(w, 0.0, None, out=w)

        loss = (delL * w).sum() + (GRAPH_LEARN_BETA / 2.0) * (w ** 2).sum() - GRAPH_LEARN_ALPHA * np.sum(np.log(Sw[nonzero]))
        if loss < best_loss:
            best_loss = loss
            best_w = w.copy()

    num_edges = len(best_w)
    k = max(1, int(0.05 * num_edges))
    threshold = np.partition(best_w, -k)[-k]
    top_mask = best_w >= threshold

    adj = np.zeros((N, N), dtype=int)
    adj[i_idx[top_mask], j_idx[top_mask]] = 1
    adj[j_idx[top_mask], i_idx[top_mask]] = 1

    return adj

# -----------------------
# LapRLS Model
# -----------------------
class LapRLS:
    def __init__(self, opt):
        self.opt = opt

    def fit(self, X, Y):
        self.X_labeled = X
        self.Y_labeled = Y.flatten()
        l = X.shape[0]

        if self.opt['neighbor_mode'] == 'learned':
            X_tensor = torch.tensor(self.X_labeled, dtype=torch.float32)
            adj = learn_graph(X_tensor, seed_graph=self.opt['graph_seed'])

            W_dense = np.exp(-cdist(self.X_labeled, self.X_labeled) ** 2 / (4 * self.opt['t']))
            W_dense *= adj  # Apply adjacency mask
            W = sparse.csr_matrix(W_dense)
        else:
            raise Exception("Only 'learned' mode supported")

        L = sparse.diags(np.array(W.sum(0))[0]).tocsr() - W
        K = self.opt['kernel_function'](self.X_labeled, self.X_labeled, **self.opt['kernel_parameters'])
        I_l = np.identity(l)

        self.alpha = np.linalg.inv(
            K + self.opt['gamma_A'] * l * I_l + (self.opt['gamma_I'] * l / (l ** 2)) * L.dot(K)
        ).dot(self.Y_labeled)

    def decision_function(self, X):
        new_K = self.opt['kernel_function'](self.X_labeled, X, **self.opt['kernel_parameters'])
        return np.squeeze(self.alpha.dot(new_K))

# -----------------------
# Hyperparameters
# -----------------------
opt = {
    'neighbor_mode': 'learned',
    't': 1,
    'gamma_A': 0.0001,
    'gamma_I': 1,
    'kernel_function': rbf,
    'kernel_parameters': {'gamma': 1},
    'graph_seed': GRAPH_SEED,
}

# -----------------------
# Load and preprocess data
# -----------------------
DATA_PATH = '1000_spherical_anomalies.csv'  # Ensure this file exists
df = pd.read_csv(DATA_PATH)
X = np.abs(df.iloc[:, :22].values)
y = df.iloc[:, -1].values
X = MinMaxScaler().fit_transform(X)

# ----------------
# Loop over seeds
# ----------------
seeds = [8, 14, 25, 56, 80]

# for seed in seeds:
#     X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.8, random_state=seed)
#     _, X_test, _, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

#     model = LapRLS(opt)
#     model.fit(X_train, y_train)
#     y_test_pred = model.decision_function(X_test)

#     r2 = r2_score(y_test, y_test_pred)
#     mse = mean_squared_error(y_test, y_test_pred)

#     print(f"Seed {seed}: R² = {r2:.4f}, MSE = {mse:.4f}")

# ----------------
# Loop over seeds
# ----------------
r2_scores = []
mse_scores = []

for seed in seeds:
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.8, random_state=seed)
    _, X_test, _, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

    model = LapRLS(opt)
    model.fit(X_train, y_train)
    y_test_pred = model.decision_function(X_test)

    r2 = r2_score(y_test, y_test_pred)
    mse = mean_squared_error(y_test, y_test_pred)

    r2_scores.append(r2)
    mse_scores.append(mse)

    print(f"Seed {seed}: R² = {r2:.4f}, MSE = {mse:.4f}")

# ----------------
# Summary Statistics
# ----------------
r2_mean = np.mean(r2_scores)
r2_std = np.std(r2_scores)
mse_mean = np.mean(mse_scores)
mse_std = np.std(mse_scores)

print("\nSummary across seeds:")
print(f"R² Mean = {r2_mean:.4f}, R² Std = {r2_std:.4f}")
print(f"MSE Mean = {mse_mean:.4f}, MSE Std = {mse_std:.4f}")




Seed 8: R² = 0.8916, MSE = 0.3496
Seed 14: R² = 0.9016, MSE = 0.3606
Seed 25: R² = 0.9015, MSE = 0.3548
Seed 56: R² = 0.8932, MSE = 0.4660
Seed 80: R² = 0.9084, MSE = 0.3244

Summary across seeds:
R² Mean = 0.8993, R² Std = 0.0062
MSE Mean = 0.3711, MSE Std = 0.0490
